In [1]:
try:
    import openmdao.api as om
except ImportError:
    !python -m pip install openmdao[notebooks]
    import openmdao.api as om

# ExplicitComponent

Explicit variables are those that are computed as an explicit function of other variables. For instance, $z$ would be an explicit variable, given $z=sin(y)$, while $y$ would not be, given that it is defined implicitly by the nonlinear equation $cos(x⋅y)−z⋅y=0$.

In OpenMDAO, explicit variables are defined by writing a class that inherits from the :ref:`ExplicitComponent <openmdao.core.explicitcomponent.py>`  class. The explicit variables would be considered outputs while the variables on which they depend would be considered inputs (e.g., $y$ in $z=sin(y)$).

## ExplicitComponent Methods

The implementation of each method will be illustrated using a simple explicit component that computes the output area as a function of inputs *length* and *width*.

In [ ]:
class RectangleComp(om.ExplicitComponent):
    """
    A simple Explicit Component that computes the area of a rectangle.
    """

* `setup()` :

Declare input and output variables via `add_input` and `add_output`. Information such as variable names, sizes, units, and bounds are declared here.

In [ ]:
def setup(self):
    self.add_input('length', val=1.)
    self.add_input('width', val=1.)
    self.add_output('area', val=1.)

* `setup_partials()` :

Declare partial derivatives that this component provides, using wild cards to say that this component provides derivatives of all outputs with respect to all inputs.

In [ ]:
def setup_partials(self):
    self.declare_partials('*', '*')

* `compute(inputs, outputs)` :

Compute the `outputs` given the `inputs`.

In [ ]:
def compute(self, inputs, outputs):
    outputs['area'] = inputs['length'] * inputs['width']

* `compute_partials(inputs, partials)` :

[Optional] Compute the `partials` (partial derivatives) given the `inputs`.

In [ ]:
def compute_partials(self, inputs, partials):
    partials['area', 'length'] = inputs['width']
    partials['area', 'width'] = inputs['length']

* `compute_jacvec_product(inputs, d_inputs, d_outputs, mode)` :

[Optional] Provide the partial derivatives as a matrix-vector product. If `mode` is 'fwd', this method must compute $d\_outputs=J⋅d\_inputs$, where J is the partial derivative Jacobian. If `mode` is 'rev', this method must compute $d\_inputs=J^T⋅d\_outputs$.

In [ ]:
def compute_jacvec_product(self, inputs, d_inputs, d_outputs, mode):
    if mode == 'fwd':
        if 'area' in d_outputs:
            if 'length' in d_inputs:
                d_outputs['area'] += inputs['width'] * d_inputs['length']
            if 'width' in d_inputs:
                d_outputs['area'] += inputs['length'] * d_inputs['width']
    elif mode == 'rev':
        if 'area' in d_outputs:
            if 'length' in d_inputs:
                d_inputs['length'] += inputs['width'] * d_outputs['area']
            if 'width' in d_inputs:
                d_inputs['width'] += inputs['length'] * d_outputs['area']

* `compute_multi_jacvec_product(self, inputs, d_inputs, d_outputs, mode)` :

[Optional] Provide the partial derivatives as a matrix-matrix product. If `mode` is 'fwd', this method must compute $d\_outputs=J⋅d\_inputs$, where $J$ is the partial derivative Jacobian, and where both $d\_outputs$ and $d\_inputs$ are matrices instead of vectors. If `mode` is 'rev', this method must similarly compute $d\_inputs=J^T⋅d\_outputs$. Note that in this case, the code in `compute_multi_jacvec_product` is the same as the code in `compute_jacvec_product`. This won’t always be the case, depending on the math operations that are required for multiplying by a matrix versus multiplying by a vector.

This method is only used when “vectorize_derivs” is set to True on a design variable or response.

In [ ]:
def compute_multi_jacvec_product(self, inputs, d_inputs, d_outputs, mode):
    if mode == 'fwd':
        if 'area' in d_outputs:
            if 'length' in d_inputs:
                d_outputs['area'] += inputs['width'] * d_inputs['length']
            if 'width' in d_inputs:
                d_outputs['area'] += inputs['length'] * d_inputs['width']
    elif mode == 'rev':
        if 'area' in d_outputs:
            if 'length' in d_inputs:
                d_inputs['length'] += inputs['width'] * d_outputs['area']
            if 'width' in d_inputs:
                d_inputs['width'] += inputs['length'] * d_outputs['area']

Note that the last three are optional, because the class can implement compute_partials, one or both of `compute_jacvec_product` and `compute_multi_jacvec_product`, or neither if the user wants to use the finite-difference or complex-step method.

In [3]:
om.display_source("openmdao.core.group.Group")

class Group(System):
    """
    Class used to group systems together; instantiate or inherit.

    Attributes
    ----------
    _mpi_proc_allocator : ProcAllocator
        Object used to allocate MPI processes to subsystems.
    _proc_info : dict of subsys_name: (min_procs, max_procs, weight)
        Information used to determine MPI process allocation to subsystems.
    _subgroups_myproc : list
        List of local subgroups.
    _manual_connections : dict
        Dictionary of input_name: (output_name, src_indices) connections.
    _group_inputs : dict
        Mapping of promoted names to certain metadata (src_indices, units).
    _static_group_inputs : dict
        Group inputs added outside of setup/configure.
    _pre_config_group_inputs : dict
        Group inputs added inside of setup but before configure.
    _static_manual_connections : dict
        Dictionary that stores all explicit connections added outside of setup.
    _conn_abs_in2out : {'abs_in': 'abs_out'}
        Dictionary containing all explicit & implicit connections owned
        by this system only. The data is the same across all processors.
    _conn_discrete_in2out : {'abs_in': 'abs_out'}
        Dictionary containing all explicit & implicit discrete var connections owned
        by this system only. The data is the same across all processors.
    _transfers : dict of dict of dict of Transfers
        First key is the vec_name, second key is mode, third is subname where
        mode is 'fwd' or 'rev' and subname is the subsystem name
        or subname can be None for the full, simultaneous transfer.
    _discrete_transfers : dict of discrete transfer metadata
        Key is system pathname or None for the full, simultaneous transfer.
    _approx_subjac_keys : list
        List of subjacobian keys used for approximated derivatives.
    _setup_procs_finished : bool
        Flag to check if setup_procs is complete
    _has_distrib_vars : bool
        If True, this Group contains distributed variables. Only used to determine if a parallel
        group or distributed component is below a DirectSolver so that we can raise an exception.
    _contains_parallel_group : bool
        If True, this Group contains a ParallelGroup. Only used to determine if a parallel
        group or distributed component is below a DirectSolver so that we can raise an exception.
    _raise_connection_errors : bool
        Flag indicating whether connection errors are raised as an Exception.
    _order_set : bool
        Flag to check if set_order has been called.
    _auto_ivc_warnings : list
        List of Auto IVC warnings to be raised later with simple_warnings.
    _shapes_graph : nx.OrderedGraph
        Dynamic shape dependency graph, or None.
    _shape_knowns : set
        Set of shape dependency graph nodes with known (non-dynamic) shapes.
    """

    def __init__(self, **kwargs):
        """
        Set the solvers to nonlinear and linear block Gauss--Seidel by default.

        Parameters
        ----------
        **kwargs : dict
            dict of arguments available here and in all descendants of this
            Group.
        """
        self._mpi_proc_allocator = DefaultAllocator()
        self._proc_info = {}

        super().__init__(**kwargs)

        self._subgroups_myproc = None
        self._manual_connections = {}
        self._group_inputs = {}
        self._pre_config_group_inputs = {}
        self._static_group_inputs = {}
        self._static_manual_connections = {}
        self._conn_abs_in2out = {}
        self._conn_discrete_in2out = {}
        self._transfers = {}
        self._discrete_transfers = {}
        self._approx_subjac_keys = None
        self._setup_procs_finished = False
        self._has_distrib_vars = False
        self._contains_parallel_group = False
        self._raise_connection_errors = True
        self._order_set = False
        self._shapes_graph = None
        self._shape_knowns = None

        # TODO: we cannot set th